## 1. Setup and Configuration

In [6]:
import os
import time
import json
import requests
from typing import Dict

# Modern data stack
import psycopg
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

# Configuration - Auto-detect environment
# Set NOETL_ENV=kubernetes to run against in-cluster services
# Set NOETL_ENV=localhost (default) to run against port-forwarded services
ENVIRONMENT = os.getenv("NOETL_ENV", "localhost").lower()

if ENVIRONMENT == "kubernetes":
    # In-cluster configuration
    DB_CONFIG = {
        "host": "postgres.postgres.svc.cluster.local",
        "port": "5432",
        "user": os.getenv("POSTGRES_USER", "demo"),
        "password": os.getenv("POSTGRES_PASSWORD", "demo"),
        "dbname": os.getenv("POSTGRES_DB", "demo_noetl")
    }
    NOETL_SERVER_URL = "http://noetl.noetl.svc.cluster.local:8082"
else:
    # Localhost configuration (port-forwarded from kind cluster)
    DB_CONFIG = {
        "host": "localhost",
        "port": "54321",  # Maps to postgres NodePort 30321
        "user": os.getenv("POSTGRES_USER", "demo"),
        "password": os.getenv("POSTGRES_PASSWORD", "demo"),
        "dbname": os.getenv("POSTGRES_DB", "demo_noetl")
    }
    NOETL_SERVER_URL = "http://localhost:8082"  # Maps to noetl NodePort 30082

TEST_PATH = "tests/pagination/loop_with_pagination/loop_with_pagination"
POLL_INTERVAL = 2
MAX_WAIT = 120

print("✓ Configuration loaded")
print(f"  Environment: {ENVIRONMENT}")
print(f"  Server: {NOETL_SERVER_URL}")
print(f"  Database: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}")
print(f"  Test: {TEST_PATH}")


✓ Configuration loaded
  Environment: localhost
  Server: http://localhost:8082
  Database: localhost:54321/demo_noetl
  Test: tests/pagination/loop_with_pagination/loop_with_pagination


## 2. Initialize Test Table

In [3]:
# Create test schema and table - completely self-contained
import os
import psycopg

def get_postgres_connection():
    """Get psycopg3 connection"""
    conn_string = f"host={os.getenv('POSTGRES_HOST', 'postgres.postgres.svc.cluster.local')} " \
                  f"port={os.getenv('POSTGRES_PORT', '5432')} " \
                  f"dbname={os.getenv('POSTGRES_DB', 'demo_noetl')} " \
                  f"user={os.getenv('POSTGRES_USER', 'demo')} " \
                  f"password={os.getenv('POSTGRES_PASSWORD', 'demo')}"
    return psycopg.connect(conn_string)

create_table_sql = """
CREATE SCHEMA IF NOT EXISTS noetl_test;

DROP TABLE IF EXISTS noetl_test.pagination_loop_results;

CREATE TABLE noetl_test.pagination_loop_results (
    id SERIAL PRIMARY KEY,
    execution_id BIGINT,
    endpoint_name TEXT,
    endpoint_path TEXT,
    page_size INTEGER,
    result_count INTEGER,
    result_data JSONB,
    iteration_index INTEGER,
    iteration_count INTEGER,
    created_at TIMESTAMP DEFAULT NOW()
);

CREATE INDEX IF NOT EXISTS idx_pagination_loop_execution_id 
ON noetl_test.pagination_loop_results(execution_id);
"""

with get_postgres_connection() as conn:
    with conn.cursor() as cur:
        cur.execute(create_table_sql)
        conn.commit()

print("✓ Test table created")
print("  Schema: noetl_test")
print("  Table: pagination_loop_results")

✓ Test table created
  Schema: noetl_test
  Table: pagination_loop_results


## 3. Database Utilities

In [4]:
def get_postgres_connection():
    """Get psycopg3 connection"""
    conn_string = f"host={DB_CONFIG['host']} port={DB_CONFIG['port']} " \
                  f"dbname={DB_CONFIG['dbname']} user={DB_CONFIG['user']} " \
                  f"password={DB_CONFIG['password']}"
    return psycopg.connect(conn_string)

def query_to_polars(query: str) -> pl.DataFrame:
    """Execute query and return as Polars DataFrame"""
    with get_postgres_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            columns = [desc[0] for desc in cur.description]
            data = cur.fetchall()
    if not data:
        return pl.DataFrame(schema=columns)
    return pl.DataFrame({col: [row[i] for row in data] for i, col in enumerate(columns)})

print("✓ Database utilities loaded")

✓ Database utilities loaded


## 4. Execute Pagination Loop Test

In [7]:
def start_test() -> Dict:
    """Start pagination loop test"""
    url = f"{NOETL_SERVER_URL}/api/run/playbook"
    payload = {"path": TEST_PATH}
    
    print(f"Starting test: {TEST_PATH}")
    response = requests.post(url, json=payload, timeout=30)
    response.raise_for_status()
    
    result = response.json()
    execution_id = result['execution_id']
    
    print(f"✓ Test started")
    print(f"  Execution ID: {execution_id}")
    print(f"  Status: {result['status']}")
    
    return result

test_result = start_test()
EXECUTION_ID = test_result['execution_id']

Starting test: tests/pagination/loop_with_pagination/loop_with_pagination
✓ Test started
  Execution ID: 511647000505352329
  Status: running


## 5. Monitor Execution

In [8]:
def monitor_execution(execution_id: int):
    """Monitor test execution"""
    start_time = time.time()
    last_count = 0
    
    print(f"Monitoring execution {execution_id}...")
    print(f"{'Time':<6} {'Steps':<6} {'Status':<12} {'Events'}")
    print("-" * 50)
    
    while (time.time() - start_time) < MAX_WAIT:
        query = f"""
            SELECT event_type, COUNT(*) as count
            FROM noetl.event
            WHERE execution_id = {execution_id}
            GROUP BY event_type
        """
        df = query_to_polars(query)
        
        step_count = df.filter(pl.col('event_type') == 'step_completed')['count'].sum() or 0
        is_complete = df.filter(pl.col('event_type') == 'playbook_completed').height > 0
        is_failed = df.filter(pl.col('event_type') == 'playbook_failed').height > 0
        
        if step_count != last_count or is_complete or is_failed:
            elapsed = int(time.time() - start_time)
            status = "COMPLETED" if is_complete else ("FAILED" if is_failed else "RUNNING")
            total = df['count'].sum()
            print(f"{elapsed:<6} {step_count:<6} {status:<12} {total}")
            last_count = step_count
        
        if is_complete:
            print(f"\n✓ Test completed in {elapsed}s")
            return True
        elif is_failed:
            print(f"\n✗ Test failed after {elapsed}s")
            return False
        
        time.sleep(POLL_INTERVAL)
    
    print(f"\n⚠ Timeout after {MAX_WAIT}s")
    return False

success = monitor_execution(EXECUTION_ID)

Monitoring execution 511647000505352329...
Time   Steps  Status       Events
--------------------------------------------------

⚠ Timeout after 120s

⚠ Timeout after 120s


## 6. Validate Iterator Event Architecture

**Expected Behavior:**
1. Worker detects `loop` configuration in step
2. Routes to iterator executor 
3. Analyzes collection (filter, sort, limit)
4. Emits `iterator_started` event with metadata
5. Server should process event and enqueue iteration jobs (NOT YET IMPLEMENTED)

This section verifies the distributed loop architecture is working correctly.

In [10]:
# Validate iterator event architecture
print(f"\n📊 Iterator Event Validation for Execution {EXECUTION_ID}")
print("=" * 80)

# 1. Check basic event flow
events_query = f"""
    SELECT 
        event_type,
        node_name,
        status,
        created_at
    FROM noetl.event
    WHERE execution_id = {EXECUTION_ID}
    ORDER BY event_id
"""
events_df = query_to_polars(events_query)

print(f"\n✓ Event Flow ({events_df.height} events):")
for row in events_df.iter_rows(named=True):
    print(f"  {row['event_type']:25} {row['status']:12} {row['node_name'] or ''}")

# 2. Check iterator_started event exists
iterator_check = f"""
    SELECT 
        event_type,
        status,
        context->>'total_count' as total_count,
        context->>'mode' as mode,
        context->>'iterator_name' as iterator_name,
        jsonb_array_length(context->'collection') as collection_size,
        context->'nested_task'->>'tool' as nested_tool
    FROM noetl.event
    WHERE execution_id = {EXECUTION_ID}
      AND event_type = 'iterator_started'
"""
iterator_df = query_to_polars(iterator_check)

if iterator_df.height > 0:
    print(f"\n✓ iterator_started Event Found:")
    row = iterator_df.row(0, named=True)
    print(f"  Status:           {row['status']}")
    print(f"  Total Count:      {row['total_count']}")
    print(f"  Collection Size:  {row['collection_size']}")
    print(f"  Mode:             {row['mode']}")
    print(f"  Iterator Name:    {row['iterator_name']}")
    print(f"  Nested Tool:      {row['nested_tool']}")
else:
    print("\n❌ iterator_started event NOT FOUND!")
    print("   This means the event callback integration failed.")

# 3. Check iterator metadata
if iterator_df.height > 0:
    metadata_query = f"""
        SELECT 
            context->'collection' as collection,
            context->'nested_task' as nested_task
        FROM noetl.event
        WHERE execution_id = {EXECUTION_ID}
          AND event_type = 'iterator_started'
    """
    
    with get_postgres_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(metadata_query)
            result = cur.fetchone()
            if result:
                collection = result[0]
                nested_task = result[1]
                
                print(f"\n✓ Iterator Metadata:")
                print(f"  Collection: {json.dumps(collection, indent=2)[:200]}...")
                print(f"\n  Nested Task Tool: {nested_task.get('tool')}")
                print(f"  Has retry.on_success: {'retry' in nested_task and 'on_success' in nested_task.get('retry', {})}")
                
                if 'retry' in nested_task and 'on_success' in nested_task['retry']:
                    retry_config = nested_task['retry']['on_success']
                    print(f"  Pagination Config:")
                    print(f"    - while: {retry_config.get('while', 'N/A')[:60]}")
                    print(f"    - max_attempts: {retry_config.get('max_attempts', 'N/A')}")
                    print(f"    - collect.strategy: {retry_config.get('collect', {}).get('strategy', 'N/A')}")

# 4. Check for expected next events (iteration jobs)
iteration_check = f"""
    SELECT COUNT(*) as count
    FROM noetl.event
    WHERE execution_id = {EXECUTION_ID}
      AND event_type = 'iteration_completed'
"""
iteration_df = query_to_polars(iteration_check)

if iteration_df['count'][0] > 0:
    print(f"\n✓ Found {iteration_df['count'][0]} iteration_completed events")
    print("  Server successfully processed iterator_started and enqueued iteration jobs!")
else:
    print(f"\n⚠ No iteration_completed events found")
    print("  This is EXPECTED - Server orchestrator doesn't yet process iterator_started")
    print("  Next Implementation Step: Add _process_iterator_started() handler in orchestrator.py")

# 5. Summary
print(f"\n{'='*80}")
print("VALIDATION SUMMARY:")
print("=" * 80)

if iterator_df.height > 0:
    print("✅ Worker Event Callback: WORKING")
    print("✅ Iterator Executor: WORKING") 
    print("✅ iterator_started Event: EMITTED")
    print("✅ Event Schema: VALID")
    print("⏳ Server Orchestrator: NOT YET IMPLEMENTED")
    print("\nNext: Implement server-side iterator_started event processing to enqueue iteration jobs")
else:
    print("❌ Iterator event architecture validation FAILED")
    print("   Check worker logs for event emission errors")


📊 Iterator Event Validation for Execution 511647000505352329

✓ Event Flow (7 events):
  playbook_started          STARTED      tests/pagination/loop_with_pagination/loop_with_pagination
  workflow_initialized      COMPLETED    workflow
  step_started              RUNNING      fetch_all_endpoints
  action_started            RUNNING      fetch_all_endpoints
  iterator_started          RUNNING      iterator
  action_completed          COMPLETED    fetch_all_endpoints
  step_result               COMPLETED    fetch_all_endpoints

✓ iterator_started Event Found:
  Status:           RUNNING
  Total Count:      2
  Collection Size:  2
  Mode:             sequential
  Iterator Name:    endpoint
  Nested Tool:      http

✓ Iterator Metadata:
  Collection: [
  {
    "name": "assessments",
    "path": "/api/v1/assessments",
    "page_size": 10
  },
  {
    "name": "users",
    "path": "/api/v1/users",
    "page_size": 15
  }
]...

  Nested Task Tool: http
  Has retry.on_success: True
  Paginatio

## 7. Architecture Status & Next Steps

**✅ Completed Implementation:**
- Worker-side event callback integration
- Iterator executor analysis and event emission  
- Event schema extensions (iterator_started, iterator_completed, etc.)
- Environment-aware configuration (localhost/kubernetes)

**⏳ Pending Implementation:**
- Server orchestrator `_process_iterator_started()` handler
- Iteration job enqueueing logic
- Iteration execution with pagination (retry.on_success)
- `iterator_completed` event emission after all iterations

**Note:** The test will timeout until server-side processing is implemented.

## Summary: Distributed Loop + Pagination Architecture

**What This Test Validates:**

This notebook demonstrates the **event-driven distributed loop architecture** where:
- Workers analyze collections and emit events
- Server processes events to orchestrate distributed execution
- Each iteration runs independently with pagination support

**Current Implementation Status:**

```
✅ PHASE 1: Worker-Side Architecture (COMPLETE)
   ├─ Loop detection in execute_task()
   ├─ Iterator executor analysis (filter, sort, limit)
   ├─ Event callback integration
   └─ iterator_started event emission

⏳ PHASE 2: Server-Side Orchestration (PENDING)
   ├─ Process iterator_started event
   ├─ Enqueue N iteration jobs
   ├─ Track iteration completion
   └─ Emit iterator_completed event

🔮 PHASE 3: Pagination via Retry (DESIGN READY)
   ├─ HTTP action executes with retry.on_success
   ├─ Server-side pagination state tracking
   ├─ Page continuation logic
   └─ Result aggregation
```

**How to Use:**

1. **Run locally:** Execute cells 1-6 (default: localhost mode)
2. **Run in-cluster:** Set `NOETL_ENV=kubernetes` before cell 2
3. **Check validation:** Cell 6 shows full architecture validation

**Next Steps:**

Implement `_process_iterator_started()` in `orchestrator.py` to:
- Parse collection from iterator_started.context
- Create N PreparedJob instances
- Enqueue to noetl.queue table
- Workers will pick up and execute with pagination